# How to generate publication quality tables


estimagic helps you generate publication quality html and LaTex tables, given a list of estimation results.

## Set up

In [1]:
import io
import re
from collections import namedtuple
from copy import copy
from copy import deepcopy

import numpy as np
import pandas as pd
import statsmodels.api as sm
from IPython.core.display import HTML
from IPython.core.display import Latex

import estimagic.visualization.estimation_table as et
from estimagic.config import TEST_DIR

In [2]:
# Load dataset
df = pd.read_csv(TEST_DIR / "visualization" / "diabetes.csv", index_col=0)

In [3]:
df.head()

,Age,Sex,BMI,ABP,S1,S2,S3,S4,S5,S6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [4]:
# Fit regressions
est = sm.OLS(endog=df["target"], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df["target"], exog=sm.add_constant(df[df.columns[0:6]])).fit()

The estimation results can be passed as `statsmodels` regression results, or as a tuple with attributes `params` (pandas DataFrame), with parameter values, standard errors and/or confidence intervals and p-values, and `info` (dict) with summary statistics of the model.

In [5]:
# Extract `params` and `info`
namedtuplee = namedtuple("namedtuplee", "params info")
est3 = namedtuplee(
    params=et._extract_params_from_sm(est),
    info={**et._extract_info_from_sm(est)},
)

# Remove redundant information
del est3.info["df_model"]
del est3.info["df_resid"]

The resulting dictionary contains all the information we need:

In [6]:
est3[0]

,value,pvalue,standard_error,ci_lower,ci_upper
const,152.133484,2.048808e-193,2.852749,146.526671,157.740298
Age,37.241211,5.616557e-01,64.117433,-88.775663,163.258084
Sex,-106.577520,8.695658e-02,62.125062,-228.678572,15.523532
BMI,787.179313,5.345260e-29,65.424126,658.594255,915.764371
ABP,416.673772,4.245663e-09,69.494666,280.088446,553.259097


In [7]:
est3[1]

{'rsquared': 0.40026108237713975,
 'rsquared_adj': 0.3947714813005003,
 'fvalue': 72.912599073987,
 'f_pvalue': 2.7007228809503304e-47,
 'dependent_variable': 'target',
 'resid_std_err': 59.97560860753489,
 'n_obs': 442.0}

In [8]:
# Make copy of estimation results
est4 = {}
est4["params"] = deepcopy(est3.params)
est4["info"] = deepcopy(est3.info)

est5 = {}
est5["params"] = deepcopy(est3.params)
est5["info"] = deepcopy(est3.info)

## Basics

Basic features include custom title and custom names for models, columns, index and parameters.

### Basic table, without title

In [9]:
ex_html = et.estimation_table([est, est2, est3, est4, est5], return_type="html")
HTML(ex_html)

,(1),(2),(3),(4),(5)
const,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Sex,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [10]:
ex_latex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    left_decimals=4,
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_latex)

<IPython.core.display.Latex object>

### Basic table, with title

In [11]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
)
HTML(ex_html)

,(1),(2),(3),(4),(5)
const,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Sex,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [12]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    left_decimals=4,
    return_type="latex",
    render_options={"caption": "This is a caption"},
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Column names

In [13]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_col_names=list("abcde"),
)
HTML(ex_html)

,a,b,c,d,e
const,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Sex,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [14]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={"caption": "This is a caption"},
    left_decimals=4,
    custom_col_names=list("abcde"),
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

Column names can be hidden by passing `show_col_names=False`:

In [15]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    show_col_names=False,
)
HTML(ex_html)

,,,,,
const,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Sex,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [16]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={"caption": "This is a caption"},
    left_decimals=4,
    show_col_names=False,
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Model names

In [17]:
custom_mod_names = {"M a": [0], "M b-d": [1, 2, 3], "M e": [4]}

In [18]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_model_names=custom_mod_names,
    custom_col_names=list("abcde"),
)
HTML(ex_html)

In [19]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={"caption": "This is a caption"},
    left_decimals=4,
    custom_model_names=custom_mod_names,
    custom_col_names=list("abcde"),
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Index name

By default, the index name is "index":

In [20]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption", "index_names": True},
    custom_model_names=custom_mod_names,
)
HTML(ex_html)

In [21]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={"caption": "This is a caption", "index_names": True},
    left_decimals=4,
    custom_model_names=custom_mod_names,
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

This can be customized by passing a different index name to `custom_index_names`:

In [22]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    custom_model_names=custom_mod_names,
)
HTML(ex_html)

In [23]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    left_decimals=4,
    custom_model_names=custom_mod_names,
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Parameter names

Custom parameter names can be specified by passing a dictionary to `custom_param_names`:

In [24]:
cust_par_names = {"const": "Intercept", "Sex": "Gender"}

In [25]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    custom_model_names=custom_mod_names,
    custom_param_names=cust_par_names,
)
HTML(ex_html)

In [26]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    left_decimals=4,
    custom_model_names=custom_mod_names,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

## Advanced

### Confidence intervals

In [27]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    custom_model_names=custom_mod_names,
    custom_param_names=cust_par_names,
    confidence_intervals=True,
)
HTML(ex_html)

In [28]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    left_decimals=4,
    custom_model_names=None,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
    confidence_intervals=True,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

Passing `confidence_intervals=False` prints standard errors. To hide both standard errors and confidence intervals you need to pass `show_inference=False`:

In [29]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_index_names=["Variables"],
    custom_model_names=custom_mod_names,
    custom_param_names=cust_par_names,
    show_inference=False,
)
HTML(ex_html)

In [30]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    custom_index_names=["Variables"],
    left_decimals=4,
    custom_model_names=None,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
    show_inference=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Footer

To avoid printint statistics, such as R-squared and number of observations, pass `show_footer=False`.

In [31]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_param_names=cust_par_names,
    show_footer=False,
)
HTML(ex_html)

,(1),(2),(3),(4),(5)
Intercept,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Gender,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [32]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    left_decimals=4,
    custom_model_names=None,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
    show_footer=False,
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Custom notes

In [33]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_param_names=cust_par_names,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
)
HTML(ex_html)

,(1),(2),(3),(4),(5)
Intercept,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Gender,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


In [34]:
ex_tex = et.estimation_table(
    [est, est2, est3, est4],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    left_decimals=4,
    custom_model_names=None,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Custom names for summary statistics

In [35]:
ex_html = et.estimation_table(
    [est, est2, est3, est4, est5],
    return_type="html",
    render_options={"caption": "This is a caption"},
    custom_param_names=cust_par_names,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
    stats_dict={"R$^2$": "rsquared", "N. Obs": "n_obs"},
)
HTML(ex_html)

,(1),(2),(3),(4),(5)
Intercept,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$,152.13$^{*** }$
,(2.85),(2.85),(2.85),(2.85),(2.85)
Age,37.24$^{ }$,24.7$^{ }$,37.24$^{ }$,37.24$^{ }$,37.24$^{ }$
,(64.12),(65.41),(64.12),(64.12),(64.12)
Gender,-106.58$^{* }$,-82.86$^{ }$,-106.58$^{* }$,-106.58$^{* }$,-106.58$^{* }$
,(62.13),(64.85),(62.13),(62.13),(62.13)
BMI,787.18$^{*** }$,789.74$^{*** }$,787.18$^{*** }$,787.18$^{*** }$,787.18$^{*** }$
,(65.42),(66.89),(65.42),(65.42),(65.42)
ABP,416.67$^{*** }$,397.58$^{*** }$,416.67$^{*** }$,416.67$^{*** }$,416.67$^{*** }$
,(69.49),(70.87),(69.49),(69.49),(69.49)


## MultiIndex

### Set up

In [36]:
# Convert `params` DataFrame to MultiIndex
df = et._extract_params_from_sm(est)
df.index = pd.MultiIndex.from_arrays(
    np.array([["Intercept", "Slope", "Slope", "Slope", "Slope"], df.index.values])
)
df

value         pvalue  standard_error    ci_lower  \
Intercept const  152.133484  2.048808e-193        2.852749  146.526671   
Slope     Age     37.241211   5.616557e-01       64.117433  -88.775663   
          Sex   -106.577520   8.695658e-02       62.125062 -228.678572   
          BMI    787.179313   5.345260e-29       65.424126  658.594255   
          ABP    416.673772   4.245663e-09       69.494666  280.088446   

                   ci_upper  
Intercept const  157.740298  
Slope     Age    163.258084  
          Sex     15.523532  
          BMI    915.764371  
          ABP    553.259097

In [37]:
# Extract info and generate tuple of estimation results for `est1`
info = et._extract_info_from_sm(est)
est_mi = namedtuplee(params=df, info=info)

In [38]:
# Repeat for `est2`
df = et._extract_params_from_sm(est2)
df.index = pd.MultiIndex.from_arrays(
    np.array(
        [
            ["Intercept", "Slope", "Slope", "Slope", "Slope", "Else", "Else"],
            df.index.values,
        ]
    )
)
info = et._extract_info_from_sm(est2)
est_mi2 = namedtuplee(params=df, info=info)

### Basics

In [39]:
ex_html = et.estimation_table(
    [est_mi, est_mi2],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_param_names=cust_par_names,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
)
HTML(ex_html)

In [40]:
ex_tex = et.estimation_table(
    [est_mi, est_mi2],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    left_decimals=3,
    custom_model_names=None,
    custom_param_names=cust_par_names,
    alignment_warning=False,
    siunitx_warning=False,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Parameter names

In [41]:
ex_html = et.estimation_table(
    [est_mi, est_mi2],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
    custom_param_names={"Age": "Maturity", "Else": "Additionally"},
)
HTML(ex_html)

In [42]:
ex_tex = et.estimation_table(
    [est_mi, est_mi2],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    left_decimals=3,
    custom_model_names=None,
    alignment_warning=False,
    siunitx_warning=False,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
    custom_param_names={"Age": "Maturity", "Else": "Additionally"},
)
Latex(ex_tex)

<IPython.core.display.Latex object>

### Index and model names

In [43]:
stats_dict = {
    "Observations": "n_obs",
    "R$^2$": "rsquared",
    "Adj. R$^2$": "rsquared_adj",
    "Residual Std. Error": "resid_std_err",
    "F Statistic": "fvalue",
    "show_dof": True,
}

In [44]:
ex_html = et.estimation_table(
    [est_mi, est_mi2, est_mi],
    return_type="html",
    render_options={
        "caption": "This is a caption",
    },
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
    custom_param_names={"Age": "Maturity", "Else": "Additionally"},
    custom_index_names=["Category", "Variable"],
    custom_model_names={"M1-2": [0, 1], "M3": [2]},
    stats_dict=stats_dict,
)
HTML(ex_html)

In [45]:
ex_tex = et.estimation_table(
    [est_mi, est_mi2],
    return_type="latex",
    render_options={
        "caption": "This is a caption",
    },
    left_decimals=4,
    alignment_warning=False,
    siunitx_warning=False,
    custom_notes=[
        "This is the first note of some length",
        "This is the second note probably of larger length",
    ],
    custom_param_names={"Age": "Maturity", "Else": "Additionally"},
    custom_index_names=["Category", "Variable"],
    custom_model_names={"M1": [0], "M2": [1]},
    stats_dict=stats_dict,
)
Latex(ex_tex)

<IPython.core.display.Latex object>